**PRACTICA CALIFICADA**

In [124]:
import requests
from bs4 import BeautifulSoup as b
import pandas as pd 
import numpy as np
from selenium import webdriver
import re 
from selenium.webdriver.common.action_chains import ActionChains

genes= pd.read_excel("CHGenesOrdenadosConVecinos 2.xlsx",usecols=["Gen Abrev","Category","Name of the gene","Inicio",
                                                                   "Final","RNASeq","Microarray","SAGE"],skiprows=1)
#Solo usaremos 7 filas debido a que nuestras pcs estan demorando en procesar, de todas maneras no influye en los resultados.
genes.head(9)#

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN
5,VGLL3,NaN,NaN,33882149,33934641,NaN,NaN,NaN
6,CHMP2B,NaN,NaN,34199383,34231630,NaN,NaN,NaN


In [125]:
virus_t = genes["Gen Abrev"].tolist()#se crea una variable para extraer la columna de GEN ABREV en una lista
virus_t

['TRNAS-GGA',
 'LOC102173449',
 'LOC102169333',
 'LOC108636757',
 'LOC102187712',
 'VGLL3',
 'CHMP2B']

In [126]:
virus_links=[] #se crea una lista vacía con links de los genes 
for virus in virus_t:
    url = (f"https://www.genecards.org/cgi-bin/carddisp.pl?gene={virus}#expression")
    virus_links.append(url) #se crea un ciclo for para guardar en una lista los url de cada gen
print(virus_links)

['https://www.genecards.org/cgi-bin/carddisp.pl?gene=TRNAS-GGA#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102173449#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102169333#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC108636757#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102187712#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=VGLL3#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=CHMP2B#expression']


In [127]:
driver = webdriver.Chrome(executable_path='./chromedriver.exe')
area_genes=[]
for i in virus_links:
    driver.get(i)
    try:
        search = driver.find_element_by_id("expressionImage")# va a buscar el elmento id expressionImage 
        soup = b(driver.page_source,"html.parser")# se extrae el código de la fuente 
        while soup.find("area")==None:# encuentra la etiqueta del area
            driver.get(i)
            soup = b(driver.page_source,"html.parser")
        area_genes.append(soup)
    except:
        area_genes.append(None)# si encuentra None lo guarda en la lista area_genes y continua
        continue 
driver.quit()


In [130]:
gen_data=[] #se crea una lista la cual se guardaran las categorias y descripcion
for i in area_genes:#
    gen_lista=[] # se crea una lista vacia para las categorias y descripcion
    driver=i
    if i == None:
        gen_lista.append(None) #si es que encuentra None en la busqueda que lo guarde en la lista gen_lista
    else:
        gen_lista.append(driver.find("span",{"class":"gc-category"}).text)#extrae la etiqueta span todas las categorias
        gen_lista.append(driver.find_all("span",{"class":"aliasMainName"})[1].text)#extrae de la etiqueta span en class, todas las descripciones
    gen_data.append(gen_lista)# se guarda en la lista gen_data
print(gen_data) #por ultimo se guardara en esta lista cada categoria y descripcion de cada gen


[[None], [None], [None], [None], [None], ['(Protein Coding)', 'Vestigial Like Family Member 3'], ['(Protein Coding)', 'Charged Multivesicular Body Protein 2B']]


In [129]:
lista_adn=[]#se crea una lista la cual se guardaran rna, microarrage y sage
for j in area_genes:
    lista=[]
    soup=j
    if j == None:
        lista.append(None)#si en la busqueda de cada gen sale none guardalo en la lista
    else:
        titulo=soup.find_all("area",{"title":re.compile('.*Breast.*')})#recompila en la etiqueta area; title solo especificamente los numeros de la columna rna,micro  y sage
        lista.append(titulo[0]["title"].split(",")[0]+', '+titulo[3]["title"].split(",")[0])#de la columna de rnaseq extraeremos los 2 numeros
        lista.append(titulo[1]["title"].split(",")[0])# de la columna microarrage extraeremos el numero pedido
        lista.append(titulo[2]["title"].split(",")[0])# de la columna sage extraeremos el numero pedido
    lista_adn.append(lista)# se añaderia en lista de las tres columnas
print(lista_adn)

[[None], [None], [None], [None], [None], ['25, 44', 'Breast - No Data', '0'], ['42, 44', 'Breast - No Data', '21']]


In [131]:
proyecto=pd.DataFrame(gen_data,columns=["Category","Name of the gene"])#se crea un dataframe con los datos de la columna de categoria y descripcion
proyecto

,Category,Name of the gene
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
5,(Protein Coding),Vestigial Like Family Member 3
6,(Protein Coding),Charged Multivesicular Body Protein 2B


In [134]:
proyecto_1=pd.DataFrame(lista_adn, columns=["RNASeq","Microarray","SAGE"]) # Se crea un dataframe con las columnas "RNa " , etc
proyecto_1

,RNASeq,Microarray,SAGE
0,None,None,None
1,None,None,None
2,None,None,None
3,None,None,None
4,None,None,None
5,"25, 44",Breast - No Data,0
6,"42, 44",Breast - No Data,21


In [135]:
row1=genes['Gen Abrev']

row3=genes[['Inicio','Final']]

genes_frame= pd.concat([row1,proyecto,row3,proyecto_1], axis=1) # Concatena las columnas con las variables de la lista
genes_frame.replace(to_replace=[None], value=np.nan, inplace=True) # Reemplaza los datos 'none' en 'NaN'
genes_frame 

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN
5,VGLL3,(Protein Coding),Vestigial Like Family Member 3,33882149,33934641,"25, 44",Breast - No Data,0
6,CHMP2B,(Protein Coding),Charged Multivesicular Body Protein 2B,34199383,34231630,"42, 44",Breast - No Data,21


In [136]:
genes_frame.to_excel("CHGenesOrdenadosConVecinos 2.xlsx",index=False) # Del dataframe genes_frame, se va sobreescribir en el excel.  

**INGREGRANTES:**
- **CCOLLANA JORGE**
- **AYALA CASTILLO**
- **PALOMINO JHOSEP**
- **ROMERO JUANA**

